Goal: solve efficiently a generic $2^n-1$ puzzle using path-search algorithms. <br>
Quality: number of actions in the solution. <br>
Cost: total number of actions evaluated. <br>
Efficiency: Quality vs Cost

<h1>A*</h1>
In this implementation of A* algorithm, I consider two possible admissable heuristic estimates: <br>
1. Manhattan distance. It considers the manhattan distance of each tile (except for 0 encoding a free space) from its current position to its position in the goal state and sums these distances up. <br>
2. Enhanced Manhattan. It is based on the previous idea, but adds a penalising factor for tiles which are in the correct row/column but in the wrong order. It's slightly more computationally expensive, but in general it allows to find the path to the goal sooner, with algorithm evaluating less states compared to A* with Manhattan distance.

In [29]:
import numpy as np
from collections import namedtuple, defaultdict
from heapq import heappush, heappop
from typing import Optional, List, Tuple 
from dataclasses import dataclass
from random import choice
from tqdm import tqdm

Action = namedtuple('Action', ['pos1', 'pos2'])

@dataclass
class Node:
    state: np.ndarray
    parent: Optional['Node']
    action: Optional[Action]
    g_cost: int  # actual cost from start to this node
    h_cost: int  # heuristic estimate to reach the goal state
    
    def f_cost(self) -> int:
        return self.g_cost + self.h_cost
    
    def __lt__(self, other):
        return self.f_cost() < other.f_cost()

def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    puzzle_dim = state.shape[0]
    actions = list()
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < puzzle_dim - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < puzzle_dim - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def init_puzzle(puzzle_dim: int) -> np.ndarray:
    RANDOMIZE_STEPS = 100_000
    state = np.array([i for i in range(1, puzzle_dim**2)] + [0]).reshape((puzzle_dim, puzzle_dim))
    for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
        state = do_action(state, choice(available_actions(state)))
    return state

def manhattan_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    # possible heuristic, sums manhattan distances of all tiles from their current position to their position in the goal state,
    # except for 0 (blank tile)
    total_distance = 0
    puzzle_dim = state.shape[0]
    
    for i in range(puzzle_dim):
        for j in range(puzzle_dim):
            if state[i, j] != 0:
                # position in the goal state of the current number
                goal_pos = np.where(goal_state == state[i, j])
                goal_x, goal_y = goal_pos[0][0], goal_pos[1][0]
                total_distance += abs(i - goal_x) + abs(j - goal_y)
                
    return total_distance

def enhanced_manhattan(state: np.ndarray, goal_state: np.ndarray) -> int:
    # possible heuristic based on manhattan distance with additional penalty for conflicts within a row or a column
    def count_conflicts(line1: np.ndarray, line2: np.ndarray) -> int:
        #counts conflicts (correct elements but in a wrong order) within a line (row or column)
        conflicts = 0
        for i in range(len(line1)):
            if line1[i] == 0: #skipping blank tile
                continue
            for j in range(i + 1, len(line1)):
                if line1[j] == 0: #skipping blank tile
                    continue
                # check if tiles are in this line also in the goal state
                if (line1[i] in line2) and (line1[j] in line2):
                    # get their target positions
                    pos_i = np.where(line2 == line1[i])[0][0]
                    pos_j = np.where(line2 == line1[j])[0][0]
                    # if they're in reverse order, count as conflict (penalised by 2 since each tile must move at least once)
                    if pos_i > pos_j:
                        conflicts += 2 
        return conflicts

    #intialising with Manhattan distance cost
    cost = manhattan_distance(state, goal_state)
    
    #conflicts in rows
    for i in range(state.shape[0]):
        cost += count_conflicts(state[i], goal_state[i])
    #conflicts in columns
    for j in range(state.shape[1]):
        cost += count_conflicts(state[:, j], goal_state[:, j])

    return cost

class GemPuzzleSolver:
    def __init__(self, initial_state: np.ndarray, goal_state: np.ndarray, 
                 heuristic_func: callable):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.puzzle_dim = initial_state.shape[0]
        self.heuristic_func = heuristic_func
        self.states_evaluated = 0
        
    def state_to_tuple(self, state: np.ndarray) -> tuple:
        #utilisied for hashin
        return tuple(state.flatten())
    
    def reconstruct_path(self, node: Node) -> List[Tuple[np.ndarray, Action]]:
        # Get the solution path if the goal state is reached
        path = []
        current = node
        while current.parent is not None:
            path.append((current.state, current.action))
            current = current.parent
        path.append((current.state, None))  # initial state added
        return path[::-1]  # reverse to get path 'start -> goal'
    
    def solve(self) -> Tuple[Optional[List[Tuple[np.ndarray, Action]]], int, int]:
        # Solving the puzzle with A* algorithm
        self.states_evaluated = 0
        
        # Initialize the start node
        start_node = Node(
            state=self.initial_state,
            parent=None,
            action=None,
            g_cost=0,
            h_cost=self.heuristic_func(self.initial_state, self.goal_state)
        )
        
        # Priority queue for open nodes
        open_set = [start_node]
        # Dictionary to track the best known g_cost for each state
        g_scores = defaultdict(lambda: float('inf'))
        g_scores[self.state_to_tuple(self.initial_state)] = 0
        # Set of closed (evaluated) nodes
        closed_set = set()
        
        while open_set:
            current = heappop(open_set)
            current_tuple = self.state_to_tuple(current.state)
            
            # skip if we've found a better path to this state
            if current_tuple in closed_set:
                continue
            self.states_evaluated += 1
            
            # do the goal check
            if (current.state == self.goal_state).all():
                path = self.reconstruct_path(current)
                return path, len(path) - 1, self.states_evaluated

            closed_set.add(current_tuple)
            
            # Generate and evaluate all possible moves
            for action in available_actions(current.state):
                new_state = do_action(current.state, action)
                new_state_tuple = self.state_to_tuple(new_state)
                
                # Skip if we've already evaluated this state
                if new_state_tuple in closed_set:
                    continue
                
                # Calculate new g_cost
                new_g_cost = current.g_cost + 1
                
                # Skip if we've found a better path to this state
                if new_g_cost >= g_scores[new_state_tuple]:
                    continue
                
                # Create new node
                new_node = Node(
                    state=new_state,
                    parent=current,
                    action=action,
                    g_cost=new_g_cost,
                    h_cost=self.heuristic_func(new_state, self.goal_state)
                )
                
                g_scores[new_state_tuple] = new_g_cost
                heappush(open_set, new_node)
        
        # No solution found
        return None, 0, self.states_evaluated

def print_solution(path: List[Tuple[np.ndarray, Action]]):
    if not path:
        print("No solution found")
        return
        
    print(f"\nSolution found. {len(path) - 1} moves required.")
    print("\nInitial state:")
    print(path[0][0])
    
    for i, (state, action) in enumerate(path[1:], 1):
        print(f"\nMove {i}:")
        if action:
            print(f"Move element from {action.pos2} to {action.pos1}")
        print(state)

In [30]:
puzzle_dim = 3
initial_state = init_puzzle(puzzle_dim)

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Randomizing: 100%|██████████| 100000/100000 [00:02<00:00, 46545.50it/s]


In [31]:
goal_state = np.array([i for i in range(1, puzzle_dim**2)] + [0]).reshape((puzzle_dim, puzzle_dim))
    
solver = GemPuzzleSolver(
    initial_state=initial_state,
    goal_state=goal_state,
    heuristic_func=enhanced_manhattan
)

solution_path, moves, states_evaluated = solver.solve()
print_solution(solution_path)
print(f"Number of states evaluated: {states_evaluated}")


Solution found. 24 moves required.

Initial state:
[[3 7 4]
 [5 0 6]
 [8 2 1]]

Move 1:
Move element from (2, 1) to (1, 1)
[[3 7 4]
 [5 2 6]
 [8 0 1]]

Move 2:
Move element from (2, 2) to (2, 1)
[[3 7 4]
 [5 2 6]
 [8 1 0]]

Move 3:
Move element from (1, 2) to (2, 2)
[[3 7 4]
 [5 2 0]
 [8 1 6]]

Move 4:
Move element from (1, 1) to (1, 2)
[[3 7 4]
 [5 0 2]
 [8 1 6]]

Move 5:
Move element from (2, 1) to (1, 1)
[[3 7 4]
 [5 1 2]
 [8 0 6]]

Move 6:
Move element from (2, 0) to (2, 1)
[[3 7 4]
 [5 1 2]
 [0 8 6]]

Move 7:
Move element from (1, 0) to (2, 0)
[[3 7 4]
 [0 1 2]
 [5 8 6]]

Move 8:
Move element from (1, 1) to (1, 0)
[[3 7 4]
 [1 0 2]
 [5 8 6]]

Move 9:
Move element from (0, 1) to (1, 1)
[[3 0 4]
 [1 7 2]
 [5 8 6]]

Move 10:
Move element from (0, 0) to (0, 1)
[[0 3 4]
 [1 7 2]
 [5 8 6]]

Move 11:
Move element from (1, 0) to (0, 0)
[[1 3 4]
 [0 7 2]
 [5 8 6]]

Move 12:
Move element from (1, 1) to (1, 0)
[[1 3 4]
 [7 0 2]
 [5 8 6]]

Move 13:
Move element from (1, 2) to (1, 1)
[[1 3 4]

In [32]:
solver = GemPuzzleSolver(
    initial_state=initial_state,
    goal_state=goal_state,
    heuristic_func=manhattan_distance
)

solution_path, moves, states_evaluated = solver.solve()
print_solution(solution_path)
print(f"Number of states evaluated: {states_evaluated}")


Solution found. 24 moves required.

Initial state:
[[3 7 4]
 [5 0 6]
 [8 2 1]]

Move 1:
Move element from (0, 1) to (1, 1)
[[3 0 4]
 [5 7 6]
 [8 2 1]]

Move 2:
Move element from (0, 2) to (0, 1)
[[3 4 0]
 [5 7 6]
 [8 2 1]]

Move 3:
Move element from (1, 2) to (0, 2)
[[3 4 6]
 [5 7 0]
 [8 2 1]]

Move 4:
Move element from (2, 2) to (1, 2)
[[3 4 6]
 [5 7 1]
 [8 2 0]]

Move 5:
Move element from (2, 1) to (2, 2)
[[3 4 6]
 [5 7 1]
 [8 0 2]]

Move 6:
Move element from (2, 0) to (2, 1)
[[3 4 6]
 [5 7 1]
 [0 8 2]]

Move 7:
Move element from (1, 0) to (2, 0)
[[3 4 6]
 [0 7 1]
 [5 8 2]]

Move 8:
Move element from (1, 1) to (1, 0)
[[3 4 6]
 [7 0 1]
 [5 8 2]]

Move 9:
Move element from (1, 2) to (1, 1)
[[3 4 6]
 [7 1 0]
 [5 8 2]]

Move 10:
Move element from (2, 2) to (1, 2)
[[3 4 6]
 [7 1 2]
 [5 8 0]]

Move 11:
Move element from (2, 1) to (2, 2)
[[3 4 6]
 [7 1 2]
 [5 0 8]]

Move 12:
Move element from (2, 0) to (2, 1)
[[3 4 6]
 [7 1 2]
 [0 5 8]]

Move 13:
Move element from (1, 0) to (2, 0)
[[3 4 6]